In [ ]:
%pip install fcx-playground

In [ ]:
import os
import shutil
import boto3
import numpy as np
import pandas as pd
import xarray as xr
import zarr

from typing import Generator

from fcx_playground.fcx_dataprocess.tiles_rad_range import RadRangeTilesPointCloudDataProcess

In [17]:
class HIWRAPTilesPointCloudDataProcess(RadRangeTilesPointCloudDataProcess):
    def _cleaning(self, data: xr.Dataset) -> xr.Dataset:
      # data extraction
      # scrape necessary data columns 
      extracted_data = data[['time', 'ref', 'lat', 'lon', 'alt', 'roll', 'pitch', 'head', 'range']]
      return extracted_data

    def _transformation(self, data: xr.Dataset) -> pd.DataFrame:
      #  transform the data to a suitable data formatting
      hour = data['time'].values
      lat = data['lat'].values
      lon = data['lon'].values
      alt = data['alt'].values # altitude of aircraft in meters
      roll = data["roll"].values
      pitch = data["pitch"].values
      head = data["head"].values
      ref = data['ref'].values #CRS radar reflectivity #2d data
      rad_range = data["range"].values # has lower count than ref
      
      # time correction and conversion:
      base_time = self._get_date_from_url(self.url)
      hour = self._add24hr(hour)
      delta = (hour * 3600).astype('timedelta64[s]') + base_time
      time = (delta - np.datetime64('1970-01-01')).astype('timedelta64[s]').astype(np.int64)

      # transform ref to 1d array and repeat other columns to match data dimension

      num_col = ref.shape[0] # number of cols
      num_row = ref.shape[1] # number of rows

      time = np.repeat(time, num_row)
      lon = np.repeat(lon, num_row)
      lat = np.repeat(lat, num_row)
      alt = np.repeat(alt, num_row)
      roll = np.repeat(roll * self.to_rad, num_row)
      pitch = np.repeat(pitch * self.to_rad, num_row)
      head = np.repeat(head * self.to_rad, num_row)
      rad_range = np.tile(rad_range, num_col)
      ref = ref.flatten()

      # curtain creation

      x, y, z = self._down_vector(roll, pitch, head)
      x = np.multiply(x, np.divide(rad_range, 111000 * np.cos(lat * self.to_rad)))
      y = np.multiply(y, np.divide(rad_range, 111000))
      z = np.multiply(z, rad_range)
      lon = np.add(-x, lon)
      lat = np.add(-y, lat)
      alt = np.add(z, alt)

      # sort by time

      sort_idx = np.argsort(time)
      lon = lon[sort_idx]
      lat = lat[sort_idx]
      alt = alt[sort_idx]
      ref = ref[sort_idx]
      time = time[sort_idx]

      # remove nan and infinite using mask (dont use masks filtering for values used for curtain creation)

      mask = np.logical_and(np.isfinite(ref), alt > 0)
      time = time[mask]
      ref = ref[mask]
      lon = lon[mask]
      lat = lat[mask]
      alt = alt[mask]


      df = pd.DataFrame(data = {
        'time': time[::20],
        'lon': lon[::20],
        'lat': lat[::20],
        'alt': alt[::20],
        'ref': ref[::20]
      })

      return df

    def _get_date_from_url(self, url: str) -> np.datetime64:
      # get date from url
      # date is in the format of YYYYMMDD
      # eg. 20190801
      date = url.split("HS3_HIWRAP_")[1].split("_")[0]
      np_date = np.datetime64('{}-{}-{}'.format(date[:4], date[4:6], date[6:]))
      return np_date

In [18]:
obj = HIWRAPTilesPointCloudDataProcess()

In [19]:
# %pip install netCDF4

In [20]:
data = obj.ingest("../../../../test_data/HS3_HIWRAP_20130925_kuinnerchirp_175902-183052_v03.nc")

In [21]:
data

<xarray.Dataset>
Dimensions:       (time: 115058, range: 353)
Coordinates:
  * range         (range) float32 74.95 149.9 224.8 ... 2.638e+04 2.646e+04
  * time          (time) float32 3.239e+05 3.239e+05 ... 3.259e+05 3.259e+05
Data variables: (12/34)
    alt           (time) float32 ...
    dopcorr       (time, range) float32 ...
    doph          (time, range) float32 ...
    dophu         (time, range) float32 ...
    dopl          (time, range) float32 ...
    doplh         (time, range) float32 ...
    ...            ...
    vacft         (time) float32 ...
    vuh           float32 ...
    vul           float32 ...
    vulh          float32 ...
    wvel          (time) float32 ...
    year          int16 ...
Attributes:
    title:        HIWRAP Data, NASA Goddard Space Flight Center
    filename:     /karldata4/tian/HS3/netcdf/2013/dist/dat/v3/hs3_hiwrap_kuin...
    experiment:   NASA HS3
    source:       created from HIWRAP binary file, Version 0.0
    comments:     Contact: lin.tian-1@nasa.gov
    Conventions:  CF-1.6
    history:      Wed Sep 23 15:42:03 2015: ncks -4 -L 4 -O /ghrc/ops/output/...
    NCO:          4.3.7

In [25]:
pre_processed_data = obj.preprocess(data)

In [26]:
point_clouds_tileset = obj.prep_visualization(pre_processed_data)

In [27]:
point_clouds_tileset

'temp/2013-09-25/zarr_point_cloud'